### Run notebook 02_LoL_predictor_Data_Preparation before running this one, as it prepares the data!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import yaml

In [2]:
with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

import wandb
wandb.login()

wandb.init(
        project="lol-predictor",
        entity="mschon",
        config=config['train'],
        group='pytorch',
        mode='online'
    )

wandb: Currently logged in as: mschon. Use `wandb login --relogin` to force relogin


In [3]:
pd.set_option('display.max_columns', 500)
DATA_FOLDER = '../data/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_csv(f"{DATA_FOLDER}/train.csv")
test = pd.read_csv(f"{DATA_FOLDER}/test.csv")
validation = pd.read_csv(f"{DATA_FOLDER}/validation.csv")

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [5]:
train.shape

(6382, 21)

In [6]:
test.shape

(1482, 21)

In [7]:
validation.shape

(1482, 21)

In [8]:
trainX = train.drop(columns={"blueWins"})
trainy = train.blueWins

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/memory.py", line 155, in start
    self.metrics_monitor.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [9]:
testX = test.drop(columns={"blueWins"})
testy = test.blueWins

In [10]:
nHidden = config['train']['inputNeurons']

model = nn.Sequential(
    nn.Linear(config["preparation"]["features_amount"], nHidden[0]),
    nn.ReLU(),
    nn.Linear(nHidden[0], nHidden[1]),
    nn.ReLU(),
    nn.Linear(nHidden[1], nHidden[2]),
    nn.ReLU(),
    nn.Linear(nHidden[2], nHidden[3]),
    nn.ReLU(),
    nn.Linear(nHidden[3], 1),
    nn.Sigmoid()
    # Softmax
)

In [11]:
trainX = torch.tensor(trainX.values)
testX = torch.tensor(testX.values)

In [12]:
trainy = torch.tensor(trainy.values)
testy = torch.tensor(testy.values)

# https://stackoverflow.com/a/60440460/12342419
trainy = trainy.type(torch.LongTensor)
testy = testy.type(torch.LongTensor)

In [13]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [14]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
trainX = trainX.to(torch.float32).to(device)
testX = testX.to(torch.float32).to(device)
trainy = trainy.to(torch.float32).to(device)
testy = testy.to(torch.float32).to(device)

In [17]:
trainy = trainy.reshape((trainy.shape[0], 1))
testy = testy.reshape((testy.shape[0], 1))

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/memory.py", line 155, in start
    self.metrics_monitor.start()
  File "/home/xschon/.local/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [18]:
model

Sequential(
  (0): Linear(in_features=20, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=16, bias=True)
  (7): ReLU()
  (8): Linear(in_features=16, out_features=1, bias=True)
  (9): Sigmoid()
)

In [19]:
model.to(device)
print(device)

cpu


In [20]:
torch.cuda.is_available()

False

In [21]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

RuntimeError: No CUDA GPUs are available

In [22]:
num_epochs = config['train']['epochs']

for n in range(num_epochs):
    model.train()
    y_pred = model(trainX)
    loss = compute_loss(y_pred, trainy)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        y_pred = model(testX)
        test_loss = compute_loss(y_pred, testy)

        if n % (num_epochs / 10) == 0:
            print(n)
            accuracy = accuracy_fn(y_true=testy, y_pred=torch.round(y_pred))
            
            print(f'Accuracy: {accuracy} - Test Loss: {test_loss} - Train Loss: {loss}')
            wandb.log({'epoch': n, 'accuracy': accuracy, 'loss_train': loss, 'loss_val': test_loss})
print(loss)

0
Accuracy: 48.51551956815115 - Test Loss: 0.6944659948348999 - Train Loss: 0.6941425204277039
50
Accuracy: 73.88663967611336 - Test Loss: 0.546493649482727 - Train Loss: 0.5535596609115601
100
Accuracy: 72.94197031039135 - Test Loss: 0.5424710512161255 - Train Loss: 0.5402418971061707
150
Accuracy: 72.80701754385966 - Test Loss: 0.5441097617149353 - Train Loss: 0.5368022918701172
200
Accuracy: 72.60458839406208 - Test Loss: 0.5474706888198853 - Train Loss: 0.5325727462768555
250
Accuracy: 71.7948717948718 - Test Loss: 0.5538581013679504 - Train Loss: 0.5276694297790527
300
Accuracy: 71.72739541160594 - Test Loss: 0.5634313225746155 - Train Loss: 0.5231130123138428
350
Accuracy: 71.18758434547908 - Test Loss: 0.5720224380493164 - Train Loss: 0.518749475479126
400
Accuracy: 70.8502024291498 - Test Loss: 0.5832787156105042 - Train Loss: 0.5159866809844971
450
Accuracy: 70.31039136302294 - Test Loss: 0.5865853428840637 - Train Loss: 0.5110063552856445
tensor(0.5080, grad_fn=<BinaryCrossEn

In [ ]:
y_pred.shape

In [ ]:
torch.round(y_pred[:10])

In [ ]:
testy[:10]

In [ ]:
def predict(model, test_loader):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_preds, all_preds_raw, all_labels

In [ ]:
torch.round(testX[0])

In [ ]:
testy

In [ ]:
len(model(testX))

In [ ]:
model(testX)[0:10]

In [ ]:
testy[:10]

In [ ]:
lol = model(testX[20:30])
print(lol)
#print(lol.flatten())
print(testy[20:30])

In [ ]:
testy[:10]

In [ ]:
predict(model, test)

## Tensorflow implementation

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
nHidden = config['train']['inputNeurons']

tf_model = keras.Sequential([
    keras.layers.Dense(units=16, activation='relu'),
    keras.layers.Dense(units=nHidden[0], activation='relu'),
    keras.layers.Dense(units=nHidden[1], activation='relu'),
    keras.layers.Dense(units=nHidden[2], activation='relu'),
    keras.layers.Dense(units=nHidden[3], activation='sigmoid')  #activation='softmax'
])

In [ ]:
tf_model.compile(optimizer='adam', 
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = tf_model.fit(trainX, trainy, epochs=10)